In [1]:
#import des librairies
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages com.datastax.spark:spark-cassandra-connector_2.11:2.3.2 pyspark-shell'
import time

In [2]:
#initiation de context de spark cassandra
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import desc

In [3]:
conf = SparkConf() \
    .setAppName("stream to cassandra") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "localhost")
sc = SparkContext(conf=conf) 
sqlContext=SQLContext(sc)

In [4]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [5]:
socket_stream = ssc.socketTextStream("localhost", 5557)

In [6]:
lines = socket_stream.window( 10 )

In [7]:
def splitTweet(message):
  return(Row(id=str(message[0]), source=str(message[1]),user_id=str(message[2]),\
             user_name=str(message[3]),user_followers_count=str(message[4])))

In [8]:
#connexion à cassandra
def saveToCassandra(row):
    
    if not row.isEmpty(): 
        
        sqlContext.createDataFrame(row).write\
        .format("org.apache.spark.sql.cassandra")\
        .mode('append')\
            .options(table="tweet", keyspace="ereputation")\
        .save()

In [9]:
#( lines.map(lambda row : splitTweet))
#( lines.flatMap( lambda text: splitTweet(text) )
#.foreachRDD( lambda rdd: saveToCassandra(rdd) ) )

lines.flatMap(lambda text: text.split("|"))\
.map(lambda message:splitTweet)\
.foreachRDD( lambda rdd: saveToCassandra(rdd)) 

In [10]:
# Start the computation
ssc.start()

In [11]:
#ssc.stop()